# NEURAL NETWORK 
## MNIST FASHION DATASET

![Visualization](assets/visualization.png)

### Goal

This project was realized during our competency building for the [MindGame](https://github.com/PoCInnovation/MindGame) project with [PoCInnovation](https://github.com/PoCInnovation)

[![PocInnovation](assets/poc.jpeg)](https://github.com/PoCInnovation)

### Dataset
Zalando Research Project: [Github](https://github.com/zalandoresearch/fashion-mnist)
<br>

> ## Why we made Fashion-MNIST
>
> The original [MNIST dataset](http://yann.lecun.com/exdb/mnist/) contains a lot of handwritten digits. Members of the AI/ML/Data Science community love this dataset and use it as a benchmark to validate their algorithms. In fact, MNIST is often the first dataset researchers try. *"If it doesn't work on MNIST, it **won't work** at all"*, they said. *"Well, if it does work on MNIST, it may still fail on others."*
>
> ### To Serious Machine Learning Researchers
>
> Seriously, we are talking about replacing MNIST. Here are some good reasons:
>
> - **MNIST is too easy.** Convolutional nets can achieve 99.7% on MNIST. Classic machine learning algorithms can also achieve 97% easily. Check out [our side-by-side benchmark for Fashion-MNIST vs. MNIST](http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/), and read "[Most pairs of MNIST digits can be distinguished pretty well by just one pixel](https://gist.github.com/dgrtwo/aaef94ecc6a60cd50322c0054cc04478)."
> - **MNIST is overused.** In [this April 2017 Twitter thread](https://twitter.com/goodfellow_ian/status/852591106655043584), Google Brain research scientist and deep learning expert Ian Goodfellow calls for people to move away from MNIST.
> - **MNIST can not represent modern CV tasks**, as noted in [this April 2017 Twitter thread](https://twitter.com/fchollet/status/852594987527045120), deep learning expert/Keras author François Chollet.


### Contributors

| Developer | Developer | Manager |
|:----------:|:----------:|:----------:|
| [<img src="https://github.com/Baragouin.png?size=85" width=85><br><sub>Timothé Medico</sub>](https://github.com/Baragouin) |  [<img src="https://github.com/agherasie.png?size=85" width=85><br><sub>Alexandru Gherasie</sub>](https://github.com/agherasie) | [<img src="https://github.com/Mikatech.png?size=85" width=85><br><sub>Mikaël Vallenet</sub>](https://github.com/Mikatech) |

### Prerequisites

Installation of missing dependencies

In [ ]:
% pip install torch torchvision numpy matplotlib tqdm

Importation of the necessary modules

In [ ]:
import time
import datetime
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm

### Creation of the *DeepLearning* model

We create a PyTorch Module to define our model

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        # Convolution layer
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.max_pool = nn.MaxPool2d(2, 2)
        # --------------------------------

        self.dropout = nn.Dropout(0.25)

        # Fully connected layer
        self.linear1 = nn.Linear(256, 120)
        self.linear2 = nn.Linear(120, 10)
        # --------------------------------

    def forward(self, x):
        # Convolution layer
        x = F.relu(self.conv1(x))
        x = self.max_pool(x)
        x = F.relu(self.conv2(x))
        x = self.max_pool(x)
        # -----------------------

        x = self.dropout(x)

        # Flatten filters
        x = x.view(-1, 256)

        # Fully connected layer
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        # -------------------------

        return x

We create the instance of our model and load CUDA if available

In [ ]:
network = NeuralNetwork()

# Load CUDA if available
if torch.cuda.is_available():
    network.cuda()
print("Use {}".format(['GPU with CUDA' if torch.cuda.is_available() else 'CPU']))

### Dataloading

Defining global macros and loading data into a `DataLoader` by *batch*

In [ ]:
# Constant values
EPOCH = 32
BATCH_SIZE = 64
LEARNING_RATE = 0.001

# Convert and normalize input images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load data
train_set = torchvision.datasets.FashionMNIST(root='data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)

# Select a batch of data
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# Define label names
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')

In [ ]:
# Load a loss calculator and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(network.parameters(), lr=LEARNING_RATE, momentum=0.9)

In [ ]:
def img_show(img):
    img = img / 2 + .5
    np_img = img.numpy()
    plt.imshow(np.transpose(np_img, (1, 2, 0)))
    plt.show()

### Training and monitoring

Training is done with CUDA if available

In [ ]:
iteration = 0
start_time = time.time()

train_accuracies = np.zeros(EPOCH)
test_accuracies = np.zeros(EPOCH)

for iteration in range(EPOCH):
    average_loss = 0.0

    print("――――――――――――――――――――――――――――――――――――――――――――――――――")

    # Training
    total = 0
    success = 0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        optimizer.zero_grad()
        output = network.forward(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        #average_loss += loss.data[0]
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        success += (predicted == labels.data).sum()
    train_accuracies[iteration] = 100.0 * success / total
    # -------------------------------------------------------------------

    # Testing
    total = 0
    success = 0
    for inputs, labels in tqdm(test_loader):
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        output = network.forward(inputs)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        success += (predicted == labels.data).sum()
    test_accuracies[iteration] = 100.0 * success / total
    # -------------------------------------------------------------------

    print(u"Epoch {}, average loss {}, train accuracy {}, test accuracy {}".format(
        iteration,
        average_loss / len(train_loader),
        train_accuracies[iteration],
        test_accuracies[iteration]
    ))

seconds = time.time() - start_time
duration = datetime.time(second=seconds)
display = duration.strftime("%Mm %Ss")

print("――――――――――――――――――――――――――――――――――――――――――――――――――")
print("Training time in seconds: {}".format(display))

Display accuracies across epochs

In [ ]:
plt.plot(np.arange(1, EPOCH + 1), train_accuracies)
plt.plot(np.arange(1, EPOCH + 1), test_accuracies)
plt.show()